# InterSystems IRIS FHIR Care-Gap & Readmission Risk EngineEnd-to-end portfolio notebook that connects to the InterSystems **IRIS for Health** FHIR Server, pulls population data, computes hospital readmission risk and guideline-style care gaps, and writes results back as FHIR Observations or IRIS Globals. The notebook is ready for Google Colab and can run against the official IRIS FHIR template locally or any open FHIR R4 endpoint.**Highlights**- IRIS FHIR server setup with Synthea loader (official template)- Python analytics: LACE-style readmission risk + diabetes and hypertension care gaps- Bidirectional write-back: create FHIR Observations and IRIS Globals via the Native Python SDK- Optional ingest of a *real* de-identified dataset (UCI Diabetes 130-US hospitals) to publish as FHIR Observations

## 1) Quickstart for Google Colab1. **Run the pip install cell** to pull required libraries (requests, pandas, fhir.resources, irisnative, seaborn).2. **Point `FHIR_BASE_URL` to your IRIS FHIR endpoint.** If you skip IRIS, the notebook will fall back to a public HAPI test server (read-only).3. **Run sections in order:** connect ➜ pull patients ➜ compute risk/gaps ➜ optionally post results ➜ optionally push a real dataset.> Tip: If you have an IRIS FHIR server running locally via Docker on your laptop, use `ssh -L` to forward `32783` to Colab and set `FHIR_BASE_URL` accordingly.

## 2) IRIS for Health FHIR Server Setup (local docker)This follows the InterSystems **iris-fhir-template** and loads Synthea data.```bash# 1) Clone template$ git clone https://github.com/intersystems-community/iris-fhir-template.git$ cd iris-fhir-template# 2) Start IRIS for Health Community Edition$ docker-compose up -d# 3) Generate Synthea patients (10 patients as example)$ ./synthea-loader.sh 10# 4) Load generated FHIR bundles into IRIS (namespace: FHIRSERVER)$ docker-compose exec iris iris session iris -U FHIRServerFHIRServer>d ##class(fhirtemplate.Setup).LoadPatientData("/data/fhir","FHIRSERVER","/fhir/r4")# 5) Verify serverhttp://localhost:32783/fhir/r4/metadatahttp://localhost:32783/swagger-ui/index.html```After this, the notebook can read/write FHIR resources at `http://localhost:32783/fhir/r4`.

## 3) Install dependencies (Colab-friendly)Run once per session.

In [ ]:
!pip -q install pandas numpy requests seaborn matplotlib tabulate fhir.resources intersystems-irisnative

## 4) ConfigurationSet the FHIR endpoint, credentials (if needed), and whether to allow write-back. The defaults work for the IRIS template. Public HAPI servers are often read-only; set `SERVER_SUPPORTS_WRITE=False` when using them.

In [ ]:
import osimport jsonimport mathfrom datetime import datetime, timedeltaimport pandas as pdimport numpy as npimport requestsfrom fhir.resources.observation import Observationfrom fhir.resources.patient import Patientfrom fhir.resources.fhirreference import FHIRReferencefrom fhir.resources.codeableconcept import CodeableConceptfrom fhir.resources.coding import Codingfrom fhir.resources.quantity import Quantityimport seaborn as snsimport matplotlib.pyplot as pltFHIR_BASE = os.getenv("FHIR_BASE_URL", "https://hapi.fhir.org/baseR4")FHIR_USERNAME = os.getenv("FHIR_USERNAME")FHIR_PASSWORD = os.getenv("FHIR_PASSWORD")SERVER_SUPPORTS_WRITE = os.getenv("FHIR_ALLOW_WRITE", "False").lower() == "true"print(f"FHIR_BASE set to: {FHIR_BASE}")print(f"Write-back enabled: {SERVER_SUPPORTS_WRITE}")

## 5) Helper functions for FHIR accessThe helpers keep requests minimal and notebook-friendly. Pagination is handled with `_count` and `link[next]` if present.

In [ ]:
HEADERS = {"Accept": "application/fhir+json"}def fhir_get(url, params=None):    auth = (FHIR_USERNAME, FHIR_PASSWORD) if FHIR_USERNAME and FHIR_PASSWORD else None    resp = requests.get(url, headers=HEADERS, params=params or {}, auth=auth)    resp.raise_for_status()    return resp.json()def fetch_bundle(resource_type, params=None, page_limit=5):    url = f"{FHIR_BASE}/{resource_type}"    bundle = fhir_get(url, params=params)    entries = bundle.get("entry", [])    pages = 1    while pages < page_limit:        next_link = next((l for l in bundle.get("link", []) if l.get("relation") == "next"), None)        if not next_link:            break        bundle = fhir_get(next_link["url"])        entries.extend(bundle.get("entry", []))        pages += 1    return entriesdef extract_id(resource):    ref = resource.get("id") or resource.get("fullUrl", "").rsplit("/", 1)[-1]    return ref

## 6) Pull a patient cohort and core resourcesWe grab Patients plus their Encounters, Conditions, and Observations (for labs/BP). Adjust `_count` to control sample size.

In [ ]:
PATIENT_COUNT = 50patients_raw = fetch_bundle("Patient", params={"_count": PATIENT_COUNT})patient_ids = [extract_id(entry.get("resource", {})) for entry in patients_raw]print(f"Fetched {len(patient_ids)} patients")encounters_by_patient = {}conditions_by_patient = {}obs_by_patient = {}for pid in patient_ids:    encounters_by_patient[pid] = [e.get("resource") for e in fetch_bundle("Encounter", params={"patient": f"Patient/{pid}", "_count": 50})]    conditions_by_patient[pid] = [c.get("resource") for c in fetch_bundle("Condition", params={"patient": f"Patient/{pid}", "_count": 50})]    obs_by_patient[pid] = [o.get("resource") for o in fetch_bundle("Observation", params={"patient": f"Patient/{pid}", "_count": 100, "category": "laboratory,vital-signs"})]print("Sample patient IDs:", patient_ids[:5])

## 7) Readmission risk (LACE-like) + care gaps- **Length of stay:** from Encounter period- **Acuity:** emergency encounters add points- **Comorbidities:** count distinct ICD-10/condition codes- **ED visits:** encounter class = EMERCare gaps:- Diabetes (ICD-10 E11*) without HbA1c lab in past 180 days- Hypertension (ICD-10 I10) without blood pressure in past 365 days

In [ ]:
def encounter_los(enc):    period = enc.get("period") or {}    start = period.get("start")    end = period.get("end") or start    if not start:        return 0    start_dt = datetime.fromisoformat(start[:19])    end_dt = datetime.fromisoformat(end[:19]) if end else start_dt    return max((end_dt - start_dt).days, 0)def lace_score(encounters, conditions):    los_points = sum(min(encounter_los(e), 14) for e in encounters)    acuity = sum(3 for e in encounters if e.get("class", {}).get("code") in {"EMER", "IMP"})    comorbidity = min(len({c.get("code", {}).get("coding", [{}])[0].get("code") for c in conditions if c.get("code")}), 6)    ed_visits = sum(1 for e in encounters if e.get("class", {}).get("code") == "EMER")    return los_points + acuity + comorbidity + ed_visitsdef care_gap_labels(pid, conditions, observations):    gaps = []    condition_codes = {c.get("code", {}).get("coding", [{}])[0].get("code", "") for c in conditions if c.get("code")}    now = datetime.utcnow()    def has_recent_obs(loinc_codes, days):        threshold = now - timedelta(days=days)        for obs in observations:            codes = [coding.get("code") for coding in obs.get("code", {}).get("coding", [])]            if not set(codes).intersection(loinc_codes):                continue            effective = obs.get("effectiveDateTime") or obs.get("issued")            if not effective:                continue            try:                effective_dt = datetime.fromisoformat(effective[:19])            except ValueError:                continue            if effective_dt >= threshold:                return True        return False    if any(code.startswith("E11") for code in condition_codes):        if not has_recent_obs({"4548-4", "17856-6"}, 180):            gaps.append("Diabetes: no HbA1c in last 6 months")    if "I10" in condition_codes:        if not has_recent_obs({"8480-6", "8462-4"}, 365):            gaps.append("Hypertension: no BP in last 12 months")    return gapsrecords = []for pid in patient_ids:    encounters = encounters_by_patient.get(pid, [])    conditions = conditions_by_patient.get(pid, [])    observations = obs_by_patient.get(pid, [])    score = lace_score(encounters, conditions)    bucket = "High" if score >= 14 else "Medium" if score >= 8 else "Low"    gaps = care_gap_labels(pid, conditions, observations)    display_name = next((p.get("resource", {}).get("name", [{}])[0].get("text", pid) for p in patients_raw if extract_id(p.get("resource", {})) == pid), pid)    records.append({        "patient_id": pid,        "name": display_name,        "readmission_risk_score": score,        "risk_bucket": bucket,        "care_gaps": gaps    })risk_df = pd.DataFrame(records)risk_df.sort_values(by="readmission_risk_score", ascending=False).head()

### Visualize risk distribution

In [ ]:
plt.figure(figsize=(6,4))sns.histplot(risk_df["readmission_risk_score"], bins=15, kde=True)plt.title("Readmission Risk Score Distribution")plt.xlabel("Score")plt.ylabel("Patients")plt.show()

## 8) Write back results as FHIR ObservationsToggle `SERVER_SUPPORTS_WRITE=True` when your IRIS endpoint allows POST. Each patient receives a new Observation with a custom code `READMIT-RISK`.

In [ ]:
def build_risk_observation(patient_id, score, bucket):    return {        "resourceType": "Observation",        "status": "final",        "category": [{            "coding": [{"system": "http://terminology.hl7.org/CodeSystem/observation-category", "code": "survey"}]        }],        "code": {            "coding": [{                "system": "http://example.org/codes/risk",                "code": "READMIT-RISK",                "display": "Readmission risk score"            }]        },        "subject": {"reference": f"Patient/{patient_id}"},        "valueQuantity": {"value": score, "unit": "score"},        "interpretation": [{            "coding": [{                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation",                "code": bucket.upper(),                "display": bucket            }]        }]    }def post_observation(obs):    if not SERVER_SUPPORTS_WRITE:        return {"warning": "Write disabled. Set SERVER_SUPPORTS_WRITE=True to POST."}    auth = (FHIR_USERNAME, FHIR_PASSWORD) if FHIR_USERNAME and FHIR_PASSWORD else None    resp = requests.post(f"{FHIR_BASE}/Observation", headers={"Content-Type": "application/fhir+json"}, json=obs, auth=auth)    resp.raise_for_status()    return resp.json()# Example: post for top 3 highest-risk patients (no-op if write disabled)top_patients = risk_df.sort_values(by="readmission_risk_score", ascending=False).head(3)write_results = []for _, row in top_patients.iterrows():    result = post_observation(build_risk_observation(row.patient_id, float(row.readmission_risk_score), row.risk_bucket))    write_results.append({"patient_id": row.patient_id, "result": result})write_results

## 9) Optional: write to IRIS Globals via Native Python SDKShows platform alignment by persisting scores in a global `^RiskIndex(patientId)=score`.

In [ ]:
USE_IRIS_GLOBALS = False  # set True when IRIS Native port is reachable (default 51773)IRIS_HOST = os.getenv("IRIS_HOST", "localhost")IRIS_PORT = int(os.getenv("IRIS_PORT", "51773"))IRIS_NAMESPACE = os.getenv("IRIS_NAMESPACE", "FHIRSERVER")IRIS_USER = os.getenv("IRIS_USERNAME", "_SYSTEM")IRIS_PASSWORD = os.getenv("IRIS_PASSWORD", "SYS")if USE_IRIS_GLOBALS:    import irisnative    connection = irisnative.createConnection(IRIS_HOST, IRIS_PORT, IRIS_NAMESPACE, IRIS_USER, IRIS_PASSWORD)    iris = irisnative.createIris(connection)    for row in risk_df.itertuples():        iris.set(float(row.readmission_risk_score), "^RiskIndex", row.patient_id)    iris.close()    print("Stored scores in ^RiskIndex global")else:    print("IRIS global write skipped (set USE_IRIS_GLOBALS=True to enable)")

## 10) Use a real de-identified dataset (UCI Diabetes 130-US hospitals)To showcase non-synthetic data, we ingest the publicly available **Diabetes 130-US hospitals** dataset (1999–2008) and publish a subset as FHIR Observations. Each row represents a hospital encounter with a `readmitted` label and labs (glucose, A1C). We will:1. Download the CSV from UCI ML repo.2. Map encounters to pseudo patients (hashed `patient_nbr`).3. Create FHIR Observation resources for A1C and glucose and POST them to the IRIS FHIR server.> Source: [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008)

In [ ]:
import hashlibfrom io import BytesIOfrom urllib.request import urlopenUCI_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip"uci_zip = BytesIO(urlopen(UCI_URL).read())uci_df = pd.read_csv(f"zip://{UCI_URL}!diabetic_data.csv")uci_df = uci_df.replace("?", np.nan)print("Rows loaded:", len(uci_df))uci_df[['patient_nbr','A1Cresult','glucose_test']].head()

### Transform and publish a small slice as FHIR ObservationsWe build Observations for A1C (`LOINC 4548-4`) and blood glucose (`LOINC 2339-0`). Only a limited sample is posted to avoid server noise.

In [ ]:
def patient_ref_from_hash(patient_nbr):    digest = hashlib.sha256(str(patient_nbr).encode()).hexdigest()[:12]    return f"Patient/{digest}"def observation_from_row(row, code, display, value_str):    try:        value = float(value_str)    except (TypeError, ValueError):        return None    return {        "resourceType": "Observation",        "status": "final",        "code": {"coding": [{"system": "http://loinc.org", "code": code, "display": display}]},        "subject": {"reference": patient_ref_from_hash(row.patient_nbr)},        "valueQuantity": {"value": value, "unit": "mg/dL" if code == "2339-0" else "%"}    }sample_rows = uci_df.head(200)uci_observations = []for _, r in sample_rows.iterrows():    if pd.notna(r.get("A1Cresult")) and r.A1Cresult not in {">200",">300"}:        obs = observation_from_row(r, "4548-4", "Hemoglobin A1c/Hemoglobin.total in Blood", r.A1Cresult)        if obs:            uci_observations.append(obs)    if pd.notna(r.get("max_glu_serum")) and r.max_glu_serum not in {">200",">300"}:        obs = observation_from_row(r, "2339-0", "Glucose [Mass/volume] in Blood", r.max_glu_serum)        if obs:            uci_observations.append(obs)print(f"Prepared {len(uci_observations)} Observations for posting")uci_write_results = []for obs in uci_observations[:50]:  # limit posts    uci_write_results.append(post_observation(obs))uci_write_results[:5]

## 11) Next steps and resume bullets- Add a Streamlit dashboard that lists top-risk patients and open care gaps.- Demonstrate IRIS Interoperability: wrap the Python risk engine as a Business Operation in a production.- Promote Observations into a SQL projection/class for BI tooling.**Resume blurb:**“Built an end-to-end FHIR Care-Gap & Readmission Risk Engine on InterSystems IRIS for Health CE. Pulled FHIR R4 patients, computed LACE-style readmission risk and diabetes/hypertension care gaps, wrote results back as FHIR Observations and IRIS Globals via the Python Native SDK, and onboarded a real de-identified UCI hospital dataset to enrich the FHIR repository.”